In [1]:
import MDAnalysis as mda
import numpy as np
import os
from utils import *


c:\Users\gbonn\anaconda3\envs\md-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# read in each of the substrate files
directory = 'substrates_initial/'
file_names = [f for f in os.listdir(directory)]
file_names = ['18.pdb']
# Defined in the array below are the distances away from ThDP atoms to substrate atoms 
# determine by the optimized structure of substrate 6 and are used to place the aka head atoms
# radii are in order of (columns) C1, N1, N2, S1 and then rows (C2, C3, O1) 
# so C1 is (0,0) Angstroms away from C2 
radii = [
    [1.539,2.562,3.389,2.880],
    [2.533,3.205,4.764,3.784],
    [2.393,2.973,2.592,3.893]
]

In [3]:
# load receptor universe and extract ThDP atoms 
receptor = mda.Universe('int1_receptor.pdb')
ThDP_residue = receptor.select_atoms("resname TPP")

# Save the updated universe to a new PDB file
output_filename = 'substrates_aligned/ThDP_alone.pdb'
ThDP_residue.atoms.write(output_filename)

ThDP_important_indexes = get_ThDP_indexes(ThDP_residue)

# get the coordinates of important atoms in ThDP
C1_coords = get_atom_position(ThDP_residue,ThDP_important_indexes['C1'])
N1_coords = get_atom_position(ThDP_residue,ThDP_important_indexes['N1'])
N2_coords = get_atom_position(ThDP_residue,ThDP_important_indexes['N2'])
S1_coords = get_atom_position(ThDP_residue,ThDP_important_indexes['S1'])

# ThDP C1, N1, N2, S1 atom coords that will be treated as sphere centers 
centers = np.array([C1_coords,N1_coords,N2_coords,S1_coords])

# we will use the average vector of S1C1 and N1C1 to get a guess of where C2 should be located
vector_S1_to_C1 = C1_coords - S1_coords
vector_N1_to_C1 = C1_coords - N1_coords
avg_vector = (vector_S1_to_C1 + vector_N1_to_C1)/2
unit_vector = avg_vector / np.linalg.norm(avg_vector)
guess_C2 = C1_coords + unit_vector * 1.54 # C2 should be located 1.54 A away from C1


c:\Users\gbonn\anaconda3\envs\md-env\Lib\site-packages\MDAnalysis\coordinates\PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
c:\Users\gbonn\anaconda3\envs\md-env\Lib\site-packages\MDAnalysis\coordinates\PDB.py:1153: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
c:\Users\gbonn\anaconda3\envs\md-env\Lib\site-packages\MDAnalysis\coordinates\PDB.py:1200: UserWarning: Found missing chainIDs. Corresponding atoms will use value of 'X'
  warnings.warn("Found missing chainIDs."


In [5]:

# iterate through all substrates to align to the int1 geometry
for curr_file_name in file_names:
    # load substrate universe
    file_start = curr_file_name.split('.')[0]
    substrate = mda.Universe(directory+curr_file_name)
    # identify the atoms that comprise the aka substrates 
    substrate_important_indexes = get_substrate_aka_indexes(substrate)
    # initial coords of C2, C3, and O1 
    initial_positions = [get_atom_position(substrate,substrate_important_indexes['C2']),get_atom_position(substrate,substrate_important_indexes['C3']),get_atom_position(substrate,substrate_important_indexes['O1'])]

    # we go through two rounds of optimization, first using the guess location of 
    # C2 as the starting position for each atom we are trying to place (C2,C3,O1) 
    initial_guess = np.hstack([guess_C2 for i in range(3)])
    C2_optimized, C3_optimized, O1_optimized = optimize_points(centers, initial_guess, radii)
    all_optimized = [C2_optimized, C3_optimized, O1_optimized]

    # get the final error for each atom's position 
    C2_err = atom_objective(C2_optimized, centers, radii[0])
    C3_err = atom_objective(C3_optimized, centers, radii[1])
    O1_err = atom_objective(O1_optimized, centers, radii[2])

    # use the atom with the minimum error for the next round of optimization
    all_errors = [C2_err,C3_err,O1_err]
    min_error_index = all_errors.index(min(all_errors))
    redo_initial_guess = np.hstack([all_optimized[min_error_index] for i in range(3)])
    C2_reoptimized, C3_reoptimized, O1_reoptimized = optimize_points(centers, redo_initial_guess, radii)
    
    print("Final points:", C2_reoptimized, C3_reoptimized, O1_reoptimized)
    final_positions = [C2_reoptimized, C3_reoptimized, O1_reoptimized]

    # Get the rotation and translation matrix from our initial substrate to our int1 geoemtry
    R, t = kabsch_algorithm(initial_positions,final_positions)
    
    # make a copy of the substrate object and update atom positions by aligning aka head atoms
    substrate_aka_aligned = substrate.copy()
    for i in range(0,len(substrate_aka_aligned.atoms.positions)):
        atom_coords = substrate_aka_aligned.atoms[i].position
        new_coords = np.dot(R, atom_coords) + t
        substrate_aka_aligned.atoms[i].position = new_coords

    #output_filename = 'substrates_aligned/' + file_start +'_aka_aligned_int1.pdb'
    #substrate_aka_aligned.atoms.write(output_filename)
    # get the updated coords for important atoms 
    C2_coords = substrate_aka_aligned.atoms.positions[substrate_important_indexes['C2']]
    O1_coords = substrate_aka_aligned.atoms.positions[substrate_important_indexes['O1']]
    C3_coords = substrate_aka_aligned.atoms.positions[substrate_important_indexes['C3']]
    R_coords =  substrate_aka_aligned.atoms.positions[substrate_important_indexes['R']]

    # target angles represents the optimized angles of C1-C2-R ,O1-C2-R ,C3-C2-R
    # we will use these to help us reposition the R group first atom 
    target_angles = [111.1,110.2,107.5]
    R_coords_opt = optimize_angles(R_coords,C1_coords,C2_coords,O1_coords,C3_coords,target_angles)

    # get a translation matrix 
    t_R = R_coords_opt-R_coords
    R_tail, t_tail = kabsch_algorithm([R_coords,C2_coords],[R_coords_opt,C2_coords])
    
    substrate_tail_atom_indexes = [i for i in range(0,len(substrate.atoms)) if i not in substrate_important_indexes.values()]
    substrate_tail_atom_indexes.append(substrate_important_indexes['R'])

    # make a copy of the substrate object and update atom positions by aligning aka head atoms
    substrate_aligned = substrate_aka_aligned.copy()
    for i in range(0,len(substrate_aligned.atoms.positions)):
        if i in substrate_tail_atom_indexes:
            atom_coords = substrate_aligned.atoms[i].position
            new_coords = np.dot(R_tail, atom_coords) + t_tail
            substrate_aligned.atoms[i].position = new_coords

    # Save the updated universe to a new PDB file
    output_filename = 'substrates_aligned/' + file_start +'_substrate.pdb'
    substrate_aligned.atoms.write(output_filename)


    # Example usage
    # Assuming you have a universe object `u`, and you want to rotate around the bond between atoms 0 and 1
    # Rotate atoms in `atom_list` by 45 degrees
    atom1_index = substrate_important_indexes['C2']  # Change as needed
    atom2_index = substrate_important_indexes['R']  # Change as needed
    substrate_tail_atom_indexes.remove(atom2_index)
    atoms_to_rotate =  substrate_tail_atom_indexes # Example list of atoms to be rotated
    rotation_angle = 90  # Angle in degrees

    rotate_atoms(substrate_aligned, atom1_index, atom2_index, atoms_to_rotate, rotation_angle)

    # Save the modified structure
    substrate_aligned.atoms.write("rotated_structure.pdb")


    merged_universe = mda.Merge(substrate_aligned.atoms,ThDP_residue.atoms)
    
    for atom in merged_universe.atoms:
        atom.residue.resid = 1
        atom.residue.resname = "INI"
        atom.record_type = "HETATM"
    merged_universe.atoms.write('substrates_aligned/' + file_start +'_ini.pdb')

NOT CONVERGED
CONVERGED
Final points: [-34.94014025 -36.07518795  22.18527961] [-34.57235262 -37.60205263  22.58615327] [-34.04679877 -35.58955324  21.27459049]
CONVERGED


c:\Users\gbonn\anaconda3\envs\md-env\Lib\site-packages\MDAnalysis\coordinates\PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
c:\Users\gbonn\anaconda3\envs\md-env\Lib\site-packages\MDAnalysis\coordinates\PDB.py:1153: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
c:\Users\gbonn\anaconda3\envs\md-env\Lib\site-packages\MDAnalysis\coordinates\PDB.py:1200: UserWarning: Found missing chainIDs. Corresponding atoms will use value of 'X'
  warnings.warn("Found missing chainIDs."
